# BigMart Sales Prediction 



**BigMart Sales Prediction practice problem**

We have train (8523) and test (5681) data set, train data set has both input and output variable(s). We need to predict the sales for test data set.

    1. Item_Identifier: Unique product ID

    2. Item_Weight: Weight of product

    3. Item_Fat_Content: Whether the product is low fat or not

    4. Item_Visibility: The % of total display area of all products in a store allocated to the particular product

    5. Item_Type: The category to which the product belongs

    6. Item_MRP: Maximum Retail Price (list price) of the product

    7. Outlet_Identifier: Unique store ID

    8. Outlet_Establishment_Year: The year in which store was established

    9. Outlet_Size: The size of the store in terms of ground area covered

    10.Outlet_Location_Type: The type of city in which the store is located

    11.Outlet_Type: Whether the outlet is just a grocery store or some sort of supermarket

    12.Item_Outlet_Sales: Sales of the product in the particulat store. This is the outcome variable to be predicted.



This is my first Notebook, I don't know whether its good or bad, but by the way all feedbacks/suggestions are welcome.  

So, let's start with importing essential libraries,

In [ ]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn import preprocessing  
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

This is because i am going to use Linear regression to predict 'Item_Outlet_Sales'

In [ ]:
lm = linear_model.LinearRegression()

In [ ]:
train = pd.read_csv('../input/bigmart-sales-data/Train.csv') ########### Reading training Data 

In [ ]:
test = pd.read_csv('../input/bigmart-sales-data/Test.csv')  ########### Reading testing Data 

Merging training and testing data as one big dataset because it saves some trouble of making same changes in both 'Train' and 'Test' separetly.
for example : if i am imputing missing values in 'Train' by 'Most Frequent Value or Mode' then i will be using Mode in 'Test' as well.

In [ ]:
train['source'] = 'train'
test['source'] = 'test'
test['Item_Outlet_Sales'] = 0
data = pd.concat([train, test], sort = False)
print(train.shape, test.shape, data.shape)

In [ ]:
data

General Info : Datset consist of 7 categorical features, 5 numerical feature and 1 addition feature we just added to keep track of 'Train' and 'Test' data set

In [ ]:
data.dtypes 


checking if there are missing values in our dataset, 

In [ ]:
data.isnull().sum()

Since 'Item_Weight' is a continues variable , I am using mean to fill missing values . Ofcourse we've got some other options too, such as median, mode etc. 

In [ ]:
data['Item_Weight']=data['Item_Weight'].fillna(data['Item_Weight'].mean())

In [ ]:
data

I am done with numbers, Let's see what graphs have to offer.
clearly we have 3 'Outlet_Location_Type' among them Tier3 has more outlets but how does
it add any value in Sales prediction. 


In [ ]:
sns.countplot(data.Outlet_Location_Type)

similarly, 'Item_Weight' has minimal effect on 'Item_Otlet_Sales' 

In [ ]:
sns.boxplot(data['Item_Weight'])

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data.Outlet_Type)

In [ ]:
plt.figure(figsize=(25,8))
sns.countplot(data.Outlet_Identifier)

In [ ]:
plt.figure(figsize=(25,8))
sns.countplot(data.Item_Type)

In [ ]:
data['Outlet_Size'] = data['Outlet_Size'].fillna(data['Outlet_Size'].value_counts().index[0])

In [ ]:
data

In [ ]:
sns.boxplot(x='Outlet_Size',y='Item_Outlet_Sales',data = data)



In [ ]:
sns.boxplot(x='Item_Outlet_Sales',y='Item_Fat_Content',data = data)

In [ ]:
plt.figure(figsize=(20,8))
sns.boxplot(y='Item_Outlet_Sales',x='Outlet_Type',hue = 'Outlet_Location_Type',data = data)

In [ ]:
data['Item_Outlet_Sales'].describe()

In [ ]:
sns.distplot(data['Item_Outlet_Sales'])

In [ ]:
print("skewness :",data['Item_Outlet_Sales'].skew())
print("kurtosis :",data['Item_Outlet_Sales'].kurt())

In [ ]:
train.columns

In [ ]:
#### Unique items in data set
data.apply(lambda x : len(x.unique()))

In [ ]:
le = LabelEncoder()
enc = OneHotEncoder()

In [ ]:
label = ['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year']
one = ['Outlet_Size','Outlet_Location_Type','Outlet_Type']

In [ ]:
for col in label:
    data[col]=le.fit_transform(data[col]) 

In [ ]:
data

In [ ]:
for col in one:
    data[col]=le.fit_transform(data[col]) 

In [ ]:
data

In [ ]:
def model(data):
    grp = data.groupby('source')
    test = grp.get_group('test')
    train = grp.get_group('train')
    train = train.drop('source',axis=1)
    test = test.drop('source',axis=1)
    Y = train.Item_Outlet_Sales
    X = train.drop('Item_Outlet_Sales',axis=1)
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state = 42)
    model = lm.fit(x_train, y_train)
    predictions = lm.predict(x_test)
    RMSE = sqrt((( y_test-predictions)**2).mean())
    return RMSE

In [ ]:
print('RMSE with raw data :',model(data))

In [ ]:
data.corr(method='pearson')##### Linear correlation

In [ ]:
rmv = ['Item_Identifier','Item_Weight','Item_Fat_Content','Item_Type','Outlet_Establishment_Year']

In [ ]:
data.drop(rmv, axis=1, inplace=True)

In [ ]:
data.columns

In [ ]:
print('RMSE after reducing some features :',model(data))

In [ ]:
def remove_outlier(data,column):
    Q3 = data[column].quantile(.75)
    Q1 = data[column].quantile(.25)
    IQR = Q3-Q1
    data = data[~((data[column] < (Q1 - 1.5 * IQR)) |(data[column] > (Q3 + 1.5 * IQR)))]
    return data
    

In [ ]:
data_o =remove_outlier(data,'Outlet_Size')

In [ ]:
data_o

In [ ]:
print('RMSE after removing outliers :',model(data_o))